# CRIC Convergence Generation

The notebook generates and saves everything for plotting: "kidney/convergence_plots/Plots.ipynb"

## Load data and train model

In [ ]:
import xgboost
import pickle
import shap
from time import time as t
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Imputer
import sklearn
import matplotlib.pyplot as pl
import numpy as np
from tqdm import tqdm
import pandas as pd
import os

X = pd.read_csv("data/CRIC_time_4yearESRD_X.csv")
y = np.loadtxt("data/CRIC_time_4yearESRD_y.csv")

name_map = {
    "CREATININE_SERUM": "Blood Creatinine (determines eGFR)",
    "PCR_URINE_COMBINED": "Urine Protein/Creatinine Ratio",
    "CYSC_CALIBRATED": "Blood Cystatin-C",
    "UPROTEIN24H": "Urine Protein",
    "SERUM_UREA_NITROGEN": "Blood Urea Nitrogen", # (mg/dL)
    "CRCL": "Creatinine clearance",
    "SERUM_ALBUMIN": "Blood albumin",
    "FGF23_CALIBRATED": "Fibroblast growth factor 23",
    "SYSTOLIC": "Systolic blood pressure",
    "ANEMIA": "Anemia",
    "STOFHLA_SCORE": "Health literacy score",
    "CBCHEMOGLOBIN": "Blood Hemoglobin",
    "HEMATOCRIT": "Hematocrit",
    "PULSE_PRESSURE": "Pulse pressure",
    "WBC": "White blood cells",
    "BIRTH_YEAR": "Year of birth"
}
mapped_feature_names = list(map(lambda x: name_map.get(x, x), X.columns))

# split by patient id
pids = np.unique(X["PID"].values)
train_pids,test_pids = train_test_split(pids, random_state=0)
strain_pids,valid_pids = train_test_split(train_pids, random_state=0)

# find the indexes of the samples from the patient ids
train_inds = np.where([p in train_pids for p in X["PID"].values])[0]
strain_inds = np.where([p in strain_pids for p in X["PID"].values])[0]
valid_inds = np.where([p in valid_pids for p in X["PID"].values])[0]
test_inds = np.where([p in test_pids for p in X["PID"].values])[0]

# create the split datasets
X = X.drop(["PID"], axis=1)
X_train = X.iloc[train_inds,:]
X_strain = X.iloc[strain_inds,:]
X_valid = X.iloc[valid_inds,:]
X_test = X.iloc[test_inds,:]
y_train = y[train_inds]
y_strain = y[strain_inds]
y_valid = y[valid_inds]
y_test = y[test_inds]

# mean impute for linear and deep models
imp = Imputer()
imp.fit(X_strain)
X_strain_imp = imp.transform(X_strain)
X_train_imp = imp.transform(X_train)
X_valid_imp = imp.transform(X_valid)
X_test_imp = imp.transform(X_test)
X_imp = imp.transform(X)

# standardize
scaler = StandardScaler()
scaler.fit(X_strain_imp)
X_strain_imp = scaler.transform(X_strain_imp)
X_train_imp = scaler.transform(X_train_imp)
X_valid_imp = scaler.transform(X_valid_imp)
X_test_imp = scaler.transform(X_test_imp)
X_imp = scaler.transform(X_imp)

# Convert to numpy
X_strain_values = X_strain.values
X_valid_values = X_valid.values

params = {
    "colsample_bytree": 0.15,
    "max_depth": 5,
    "reg_alpha": 0,
    "reg_lambda": 0,
    "subsample": 0.7
}
xgb_model = xgboost.XGBClassifier(
    max_depth=params["max_depth"],
    n_estimators=10000,
    learning_rate=0.005,#math.pow(10, params["learning_rate"]),
    subsample=params["subsample"],
    reg_lambda=params["reg_lambda"],
    colsample_bytree=params["colsample_bytree"],
    reg_alpha=params["reg_lambda"],
    n_jobs=16,
    random_state=1
)
xgb_model.fit(
    X_strain_values, y_strain, verbose=100,
    eval_set=[(X_valid_values, y_valid)],
    eval_metric="logloss",
    early_stopping_rounds=200
)
min_valid_loss = np.min(xgb_model.evals_result_["validation_0"]["logloss"])
pickle.dump(xgb_model, open("xgb_model_final.pickle.dat", "wb"))

xgb_model = pickle.load(open("xgb_model_final.pickle.dat", "rb"))

## Get tree shap explanations

In [ ]:
# Parallelize across samples and save to files - then compose into a single numpy array
import multiprocessing
import time
import random

def explain(ind,N):
    lb = int((X_strain.values.shape[0]/N) * ind)
    ub = int((X_strain.values.shape[0]/N) * (ind+1))
    print("Working on interval:",lb,ub)
    indep_expl = shap.TreeExplainer(xgb_model, X_strain.values, feature_dependence="independent")
    phi = indep_expl.shap_values(X_strain.values[lb:ub,:])
    np.save("scratch/CRIC_indep_shap_values_{}.npy".format(ind), phi)

def save_indep_tree_shap_values(N=50):
    processes = []
    for i in range(N):
        proc = multiprocessing.Process(target=explain, args=(i,N,))
        processes.append(proc)
        proc.start()

    for one_process in processes:
        one_process.join()

    print("Done!")

    CRIC_indep_shap_values = []
    for i in range(N):
        CRIC_indep_shap_values.append(np.load("scratch/CRIC_indep_shap_values_{}.npy".format(i)))
    np.save("CRIC_indep_shap_values_final.npy", np.concatenate(CRIC_indep_shap_values,0))

save_indep_tree_shap_values()

In [ ]:
CRIC_indep_shap_values = np.load("CRIC_indep_shap_values_final.npy")
print("Independent Tree SHAP - top 10\n"+"-"*20)
print(np.array(list(X_strain.columns.values))[np.argsort(-np.abs(CRIC_indep_shap_values.sum(0)))[0:10]])

In [ ]:
# 10th biggest
one_cutoff = np.sort(np.mean(np.abs(CRIC_indep_shap_values),0))[-10]*.01
fiv_cutoff = np.sort(np.mean(np.abs(CRIC_indep_shap_values),0))[-10]*.05
ten_cutoff = np.sort(np.mean(np.abs(CRIC_indep_shap_values),0))[-10]*.1
print("10% Cutoff (10% of tenth largest global shap value):",round(ten_cutoff,6))
print("5% Cutoff (5% of tenth largest global shap value)  :",round(fiv_cutoff,6))
print("1% Cutoff (1% of tenth largest global shap value)  :",round(one_cutoff,6))

In [ ]:
np.random.seed(20)
X_values = X_strain_values
X_values[np.isnan(X_values)] = 0

x_strain_mean = np.nanmean(X_strain_values,0).reshape([1,X_strain_values.shape[1]])
mismatch_inds = np.array(range(0,6064))[xgb_model.predict(X_strain_values)!=xgb_model.predict(x_strain_mean)]
x_inds = np.random.choice(mismatch_inds, 10, replace = False)
np.argsort(xgb_model.predict(X_strain_values)!=xgb_model.predict(x_strain_mean))[-11:-1]
ten_x = X_strain_values[x_inds,:]
# Make sure all ten differ to the mean in terms of the model prediction
assert np.sum(xgb_model.predict(x_strain_mean)[0] != xgb_model.predict(ten_x)) == 10

## Run and save sampling based results (kernel and IME)

In [ ]:
import os
## Sample Explainer
for x_ind in x_inds:
    ns_lst = np.power(10,[i/2 for i in range(3*2,7*2)])
    num_iter = 10
    
    print("x_ind",x_ind)
    directory = "samplev2_{}/".format(x_ind)
    if not os.path.exists(directory): os.makedirs(directory)
    
    f = lambda inp : xgb_model.predict(inp, output_margin=True)
    se = shap.SamplingExplainer(f, x_strain_mean)
    ke = shap.KernelExplainer(f, x_strain_mean)
    x = X_strain_values[x_ind:x_ind+1,:]
    
    expl_tree = shap.TreeExplainer(xgb_model, x_strain_mean, feature_dependence="independent", model_output="margin")
    itshap = expl_tree.shap_values(x)
    
    for ns in ns_lst:
        mspf_lst = [10, 50, 100, 500, 1000, ns]
        for mspf in mspf_lst:
            # Make sure number of samples is divisible by 2
            ns = int(ns)
            if (int(ns)%2 == 1): ns = ns+1

            print("ns",ns,"mspf",mspf)

            # Time how long it takes to obtain shap_values
            s = t()
            se_out = np.vstack([se.shap_values(x, silent=True, nsamples=ns, 
                                               min_samples_per_feature=int(mspf)) for i in range(num_iter)])
            runtime = (t() - s)/float(num_iter)
            assert np.abs(se_out[0].sum() - itshap.sum()) < .01
            np.save(directory+"se_out_ns:{}_mspf:{}_runtime:{}".format(ns,mspf,runtime), se_out)

    l1reg_lst = [0.0, "aic", "bic"]
    for ns in ns_lst:
        for l1reg in l1reg_lst:
            # Make sure number of samples is divisible by 2
            ns = int(ns)
            if (int(ns)%2 == 1): ns = ns+1
        
            print("ns",ns,"l1reg",l1reg)

            # Time how long it takes to obtain shap_values
            s = t()
            ke_out = np.vstack([ke.shap_values(x, silent=True, nsamples=ns, 
                                               l1_reg=l1reg) for i in range(num_iter)])
            runtime = (t() - s)/float(num_iter)
            assert np.abs(ke_out[0].sum() - itshap.sum()) < .01
            np.save(directory+"ke_out_ns:{}_l1reg:{}_runtime:{}".format(ns,l1reg,runtime), ke_out)

### Load and compute deviations

In [ ]:
se_abs_dev_from_truth_mean_single_dict = {}
se_abs_dev_from_truth_max_single_dict = {}
indep_expl_single_ref = shap.TreeExplainer(xgb_model, x_strain_mean, feature_dependence="independent")
for x_ind in x_inds:    
    single_phi_x = indep_expl_single_ref.shap_values(X_strain.values[x_ind:x_ind+1,:])
    phi_x = CRIC_indep_shap_values[x_ind:x_ind+1,:]
    directory = "samplev2_{}/".format(x_ind)
    ns_lst_pwrs = [i/2 for i in range(3*2,7*2)]
    ns_lst = np.power(10,ns_lst_pwrs)
    se_out_files = [f for f in os.listdir(directory) if "se_out" in f]
    for mspf in [10, 50, 100, 500, 1000, "ns"]:
        abs_dev_from_truth_single_mean_lst = []
        abs_dev_from_truth_single_max_lst = []
        for ns in ns_lst:
            if mspf == "ns":
                se_out_file = [f for f in se_out_files if "ns:{}_".format(r2(ns)) in f 
                               and "mspf:{}_".format(ns) in f]
            else:
                se_out_file = [f for f in se_out_files if "ns:{}_".format(r2(ns)) in f 
                               and "mspf:{}_".format(mspf) in f]
            if not se_out_file: continue
            se_out_file = se_out_file[0]
            se_out = np.load(directory+se_out_file)
            assert np.abs(se_out[0].sum() - single_phi_x.sum()) < 0.01
            
            abs_dev_from_truth_single = np.abs(se_out-single_phi_x).mean(0)
            abs_dev_from_truth_single_mean_lst.append(abs_dev_from_truth_single.mean())
            abs_dev_from_truth_single_max_lst.append(abs_dev_from_truth_single.max())            

        if (mspf not in se_dev_from_truth_mean_dict):
            se_abs_dev_from_truth_mean_single_dict[mspf] = np.array(abs_dev_from_truth_single_mean_lst)/len(x_inds)
            se_abs_dev_from_truth_max_single_dict[mspf] = np.array(abs_dev_from_truth_single_max_lst)/len(x_inds)
        else:
            se_abs_dev_from_truth_mean_single_dict[mspf] += np.array(abs_dev_from_truth_single_mean_lst)/len(x_inds)
            se_abs_dev_from_truth_max_single_dict[mspf] += np.array(abs_dev_from_truth_single_max_lst)/len(x_inds)

sample_expl_abs_dev_single_ref_truth_mean_dict = se_abs_dev_from_truth_mean_single_dict
sample_expl_abs_dev_single_ref_truth_max_dict = se_abs_dev_from_truth_max_single_dict
pickle.dump( sample_expl_abs_dev_single_ref_truth_mean_dict, open( "sample_expl_abs_dev_single_ref_truth_mean_dict.p", "wb" ) )
pickle.dump( sample_expl_abs_dev_single_ref_truth_max_dict, open( "sample_expl_abs_dev_single_ref_truth_max_dict.p", "wb" ) )

In [ ]:
ke_abs_dev_from_truth_mean_single_dict = {}
ke_abs_dev_from_truth_max_single_dict = {}
l1reg_lst = [0.0, "aic", "bic"]

indep_expl_single_ref = shap.TreeExplainer(xgb_model, x_strain_mean, feature_dependence="independent")
for x_ind in x_inds:
    single_phi_x = indep_expl_single_ref.shap_values(X_strain.values[x_ind:x_ind+1,:])
    phi_x = CRIC_indep_shap_values[x_ind:x_ind+1,:]
    directory = "samplev2_{}/".format(x_ind)
    print(directory)
    ns_lst_pwrs = [i/2 for i in range(3*2,7*2)]
    ns_lst = np.power(10,ns_lst_pwrs)
    ke_out_files = [f for f in os.listdir(directory) if "ke_out" in f]
    
    for l1reg in l1reg_lst:
        abs_dev_from_truth_single_mean_lst = []
        abs_dev_from_truth_single_max_lst = []
        for ns in ns_lst:
            ke_out_file = [f for f in ke_out_files if "ns:{}_".format(r2(ns)) in f 
                           and "l1reg:{}_".format(l1reg) in f]
            if not ke_out_file: continue
            ke_out_file = ke_out_file[0]
            ke_out = np.load(directory+ke_out_file)
            assert np.abs(ke_out[0].sum() - single_phi_x.sum()) < 0.01
            
            abs_dev_from_truth_single = np.abs(ke_out-single_phi_x).mean(0)
            abs_dev_from_truth_single_mean_lst.append(abs_dev_from_truth_single.mean())
            abs_dev_from_truth_single_max_lst.append(abs_dev_from_truth_single.max())                        
            
        if (l1reg == 0.0): print(max_std_lst)
            
        if (l1reg not in ke_dev_from_truth_mean_dict):
            ke_abs_dev_from_truth_mean_single_dict[l1reg] = np.array(abs_dev_from_truth_single_mean_lst)/len(x_inds)
            ke_abs_dev_from_truth_max_single_dict[l1reg] = np.array(abs_dev_from_truth_single_max_lst)/len(x_inds)
        else:
            ke_abs_dev_from_truth_mean_single_dict[l1reg] += np.array(abs_dev_from_truth_single_mean_lst)/len(x_inds)
            ke_abs_dev_from_truth_max_single_dict[l1reg] += np.array(abs_dev_from_truth_single_max_lst)/len(x_inds)

kernel_expl_abs_dev_single_ref_truth_mean_dict = ke_abs_dev_from_truth_mean_single_dict
kernel_expl_abs_dev_single_ref_truth_max_dict = ke_abs_dev_from_truth_max_single_dict
pickle.dump( kernel_expl_abs_dev_single_ref_truth_mean_dict, open( "kernel_expl_abs_dev_single_ref_truth_mean_dict.p", "wb" ) )
pickle.dump( kernel_expl_abs_dev_single_ref_truth_max_dict, open( "kernel_expl_abs_dev_single_ref_truth_max_dict.p", "wb" ) )